In [37]:
import sys

sys.path.append("../")

In [38]:
import requests
import time
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from stats_arb.tests import adf_test, kpss_test, cal_half_life, pp_test
from datetime import datetime, timedelta
from ta.volatility import BollingerBands

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.ioff()

## CONFIG

In [39]:
API_BASE = 'https://fapi.binance.com/fapi/v1/'
# DATA_PATH = '/mnt/c/Users/vmodg/source/repos/BeanRepos/crypto-pair-trading/data/'
TIMEFRAME = '1d'
lookback = 3000

# TIMEFRAME = '1h'
# lookback = 30*4

# TIMEFRAME = '15m'
# lookback = 30*2


DATA_PATH = f'/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto/'


In [40]:


LABELS = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

DROP_COLUMNS=[
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]


def get_batch(symbol, interval='1m', start_time=0, limit=400):
    """Use a GET request to retrieve a batch of candlesticks. Process the JSON into a pandas
    dataframe and return it. If not successful, return an empty dataframe.
    """

    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': limit
    }
    try:
        # timeout should also be given as a parameter to the function
        response = requests.get(f'{API_BASE}klines', params, timeout=30)
    except requests.exceptions.ConnectionError:
        print('Connection error, Cooling down for 5 mins...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    except requests.exceptions.Timeout:
        print('Timeout, Cooling down for 5 min...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    if response.status_code == 200:
        return pd.DataFrame(response.json(), columns=LABELS)
    
    print(f'Got erroneous response back {symbol}: {response}. {response.text}')
    return pd.DataFrame([])


def get_candles(base, quote, start_date: datetime, end_date=None, interval='1m'):
    batches = []

    if end_date is None:
        end_date = datetime.utcnow()

    last_timestamp = int(start_date.timestamp()) * 1000
    # gather all candlesticks available, starting from the last timestamp loaded from disk or 0
    # stop if the timestamp that comes back from the api is the same as the last one
    previous_timestamp = None

    while previous_timestamp != last_timestamp:
        # stop if we reached data from today
        if datetime.fromtimestamp(last_timestamp / 1000) >= end_date:
            break

        previous_timestamp = last_timestamp

        new_batch = get_batch(
            symbol=base + quote,
            interval=interval,
            start_time=last_timestamp
        )

        # requesting candles from the future returns empty
        # also stop in case response code was not 200
        if new_batch.empty:
            break

        last_timestamp = new_batch['open_time'].max()

        # sometimes no new trades took place yet on date.today();
        # in this case the batch is nothing new
        if previous_timestamp == last_timestamp:
            break

        batches.append(new_batch)
        last_datetime = datetime.fromtimestamp(last_timestamp / 1000)

        covering_spaces = 20 * ' '
        print(datetime.now(), base, quote, interval, str(last_datetime) + covering_spaces, end='\r', flush=True)

    if len(batches) > 0:
        # write clean version of csv to parquet
        df = pd.concat(batches, ignore_index=True)
        df.drop(columns=DROP_COLUMNS, inplace=True)
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df.set_index('open_time', inplace=True)
        df = df[~df.index.duplicated(keep='first')]
        return df



In [41]:
# import requests

# url = "https://fapi.binance.com/fapi/v1/exchangeInfo"

# payload={}
# headers = {}

# response = requests.request("GET", url, headers=headers, data=payload).json()

# symbols = [s['symbol'] for s in response['symbols'] if s['contractType'] == 'PERPETUAL' and s['quoteAsset'] == 'USDT']
# # symbols[100:]
# symbols = symbols[:50]
# len(symbols)


In [42]:
symbols = [
    'BTC',
    'ETH',
    'BNB',
    'SOL',
    'ADA',
    'XRP',
    'DOT',
    'AVAX',
    'SHIB',
    'LINK',
    'TRX',
    'UNI',
    'MATIC',
    'ICP',
    'LTC',
    'NEAR',
    'APT',
    'FIL',
    'ETC',
    'ATOM',
    'WIF',
    'ARB',
    'XLM',
    'MKR',
    'OP',
    'PEPE',
    'INJ',
    'FET',
    'THETA',
    'FTM',
    'LDO',
    'SUI',
    'TIA',
    'GALA',
    'ALGO',
    'SEI',
    'AAVE',
    'BONK',
    'DYDX',
    'AEVO',
    'STRK',
    'ORDI',
    'WLD',
    'CAKE',
    '1000SATS',
    'RUNE',
    'JUP',
]

## L1

In [43]:
l1_symbols = ['BTC', 'ETH', 'BNB', 'SOL', 'ADA', 'AVAX', 'BCH', 'TRX', 'NEAR', 'APT', 'FIL', 'ATOM', 'INJ', 'FTM', 'SUI', 'XMR', 'ALGO', 'GALA', 'SEI', '']

## Smart Contract Platform

In [44]:
smc_symbols = ['ETH', 'BNB', 'SOL', 'ADA', 'AVAX', 'BCH', 'DOT', 'LINK', 'TRX', 'ICP', 'MATIC', 'NEAR', 'APT', 'ETC', 'IMX', 'XLM', 'VET', 'INJ', 'THETA', 'FTM', 'ALGO', 'IOTA', '1000LUNC', 'QTUM', 'ZETA', 'AEVO', 'WAVES']

## DeFi

In [45]:
defi_symbols = ['LINK', 'UNI', 'MKR', 'RUNE', 'LDO', 'JUP', 'AAVE', 'DYDX', 'ENA', 'PYTH', 'CAKE', 'WOO', 'CRV', '1INCH', 'JTO', 'COMP', 'PENDLE', 'GMX', 'SHUSHI', 'C98']

In [ ]:
gamefi_symbols = ['IMX', 'GALA', 'FLOKI', 'AXS', 'SAND', 'RONIN', 'ILV', 'ENJ', 'PIXEL']

In [46]:

data = []
start_time = datetime.utcnow() - timedelta(days=lookback)

for symbol in defi_symbols:
    try:
        df = pd.read_csv(f'{DATA_PATH}{TIMEFRAME}/{symbol}-USDT.csv', parse_dates=['open_time'], index_col=['open_time'])
        df = df[df.index >= start_time].copy()
    except FileNotFoundError:
        df = pd.DataFrame([], columns=['open', 'high', 'low', 'close', 'volume'])
    # print('start time', df.index[-1].to_pydatetime())

    df1 = get_candles(
        base=symbol, 
        quote='USDT', 
        start_date=df.index[-1].to_pydatetime() if len(df) > 0 else start_time,
        # end_date=datetime.now() - timedelta(days=15),
        interval=TIMEFRAME
    )
    if df1 is not None:
        df = pd.concat([df, df1])
        df = df[~df.index.duplicated(keep='first')]

    df.rename(columns={'close': symbol}, inplace=True)
    # the data is too long, just limit to recent period
    log = np.log(df[symbol].astype(np.float32))
    data.append(log)

df = pd.concat(data, axis=1)
df = df.dropna(axis=1, how='all')
# df.dropna(inplace=True, how='any')

# our of sample
# df = df[df.index <= datetime.now() - timedelta(days=15)].copy()

df.tail()

Got erroneous response back SHUSHIUSDT: <Response [400]>. {"code":-1121,"msg":"Invalid symbol."}


,LINK,UNI,MKR,RUNE,LDO,JUP,AAVE,DYDX,ENA,PYTH,CAKE,WOO,CRV,1INCH,JTO,COMP,PENDLE,GMX,C98
2024-03-30 00:00:00,2.941962,2.535283,8.230631,2.127279,1.024894,0.406398,4.820604,1.218763,NaN,-0.082295,1.524512,-0.829654,-0.379797,-0.497745,1.158986,4.341335,1.518835,3.803324,-0.890136
2024-03-31 00:00:00,2.954546,2.561790,8.276471,2.165963,1.087652,0.556239,4.855851,1.233726,NaN,-0.003205,1.533010,-0.713187,-0.363843,-0.480620,1.410426,4.376386,1.706837,3.824939,-0.871078
2024-04-01 00:00:00,2.911100,2.519791,8.228765,2.104378,1.043029,0.508743,4.863681,1.182034,NaN,-0.063579,1.456054,-0.685179,-0.423120,-0.512828,1.483217,4.326646,1.674270,3.776661,-0.935983
2024-04-02 00:00:00,2.888704,2.425599,8.230790,2.031039,0.955934,0.450757,4.749617,1.123630,-0.258771,-0.173520,1.367112,-0.831881,-0.495937,-0.612489,1.393171,4.259435,1.656780,3.683113,-1.005395
2024-04-03 00:00:00,2.903069,2.419034,8.220995,2.049498,0.981966,0.516112,4.773393,1.144223,-0.081210,-0.122507,1.396245,-0.814118,-0.479650,-0.591132,1.486615,4.271793,1.618061,3.708682,-0.991014


In [47]:
p = 1
COINTEGRATION_CONFIDENCE_LEVEL = 90

# the 90%, 95%, and 99% confidence levels for the trace statistic and maximum 
# eigenvalue statistic are stored in the first, second, and third column of 
# cvt and cvm, respectively
confidence_level_cols = {
    90: 0,
    95: 1,
    99: 2
}
confidence_level_col = confidence_level_cols[COINTEGRATION_CONFIDENCE_LEVEL]


def test_johansen(symbol_pairs):
    df_t = df[symbol_pairs].copy()
    df_t.dropna(inplace=True)

    # The second and third parameters indicate constant term, with a lag of 1. 
    result = coint_johansen(df_t, 0, p)

    trace_crit_value = result.cvt[:, confidence_level_col]
    eigen_crit_value = result.cvm[:, confidence_level_col]
#     print("trace_crit_value",trace_crit_value)
#     print("eigen_crit_value",eigen_crit_value)
#     print("lr1",result.lr1)
#     print("lr2",result.lr2)

    # The trace statistic and maximum eigenvalue statistic are stored in lr1 and lr2;
    # see if they exceeded the confidence threshold
    if np.all(result.lr1 >= trace_crit_value) and np.all(result.lr2 >= eigen_crit_value):
        # print(f"{symbol_pairs} are cointegrated")
        # The first i.e. leftmost column of eigenvectors matrix, result.evec, contains the best weights.
        v1= result.evec[:,0:1]
        hr=v1/-v1[1] #to get the hedge ratio divide the best_eigenvector by the negative of the second component of best_eigenvector
        #the regression will be: close of symbList[1] = hr[0]*close of symbList[0] + error
        #where the beta of the regression is hr[0], also known as the hedge ratio, and
        #the error of the regression is the mean reverting residual signal that you need to predict, it is also known as the "spread"
        #the spread = close of symbList[1] - hr[0]*close of symbList[0] or alternatively (the same thing):
        #do a regression with close of symbList[0] as x and lose of symbList[1] as y, and take the residuals of the regression to be the spread.
        coint_pair = dict(hedge_ratio=v1[:, 0])
        for i, s in enumerate(symbol_pairs):
            coint_pair[f'sid_{i+1}'] = s

        cointegrating_pairs.append(coint_pair)



In [48]:
import traceback


def calculate_spread(df, coint_df, selected_row, hedge_ratio, nb_symbols=2):
    spread = None
    for i in range(nb_symbols):
        if spread is None:
            spread = df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
        else:
            spread += df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
    
    spread.dropna(inplace=True)
    return spread


critical_val = 0.02

def find_stationary_portfolio(coint_df):
    data = []
    for i, _ in coint_df.iterrows():
        try:
            hedge_ratio = coint_df.iloc[i]['hedge_ratio']
            _df = df.copy()
            spread = calculate_spread(_df, coint_df, i, hedge_ratio)
            p_val = adf_test(spread, verbose=False)
            if p_val < critical_val:
                half_life = cal_half_life(spread)
                pairs_name = coint_df[[col for col in coint_df.columns if 'sid' in col]].iloc[i].values
                # print(i, pairs_name, 'is stationary with half life', half_life)
                # print(' ')
                data.append({
                    'i': i,
                    'pairs': pairs_name,
                    'half_life': half_life
                })
        except:
            print(coint_df.iloc[i])
            # traceback.print_exc()
    
    return pd.DataFrame(data)


# find_stationary_portfolio(coint_df)

In [49]:

import itertools as it

nb_symbols = 2
cointegrating_pairs = []

#get symbol pairs
pairs = list(it.combinations(symbols, nb_symbols))

for pair in pairs:
    try:
        test_johansen(list(pair))
    except Exception:
        pass

coint_df = pd.DataFrame(cointegrating_pairs)
coint_df['i'] = coint_df.index
coint_df.head()

,hedge_ratio,sid_1,sid_2,i
0,"[4.789300690919915, -4.882485457877565]",UNI,AAVE,0
1,"[4.491797448631916, -5.4009127311854055]",LDO,DYDX,1
2,"[5.3840570405874075, -3.443989781899418]",AAVE,DYDX,2
3,"[12.98571902084127, -7.8590529673697755]",AAVE,CAKE,3
4,"[8.651359945042119, -1.366248641731146]",AAVE,JUP,4


In [50]:
coint_df.to_csv(f'coint_df_{TIMEFRAME}.csv')


# Result

In [51]:
import warnings

warnings.filterwarnings("ignore")

stationary_df = find_stationary_portfolio(coint_df)
stationary_df = pd.merge(stationary_df, coint_df, on='i')
stationary_df.dropna(inplace=True)
stationary_df['i'] = stationary_df.i.astype('int64')
stationary_df['half_life'] = stationary_df.half_life.astype('int64')
if len(stationary_df) > 0:
    stationary_df.sort_values(by=['half_life'], inplace=True)
stationary_df


,i,pairs,half_life,hedge_ratio,sid_1,sid_2
1,1,"[LDO, DYDX]",16,"[4.491797448631916, -5.4009127311854055]",LDO,DYDX
2,2,"[AAVE, DYDX]",24,"[5.3840570405874075, -3.443989781899418]",AAVE,DYDX
0,0,"[UNI, AAVE]",34,"[4.789300690919915, -4.882485457877565]",UNI,AAVE


In [52]:
stationary_df.to_csv(f'stationary_df_{TIMEFRAME}.csv', index=False)
# stationary_df

In [53]:
ignored_symbols= []
mask = ~(stationary_df['sid_1'].isin(ignored_symbols) | stationary_df['sid_2'].isin(ignored_symbols))
filtered_df = stationary_df[mask]
# filtered_df['hdiff'] = filtered_df['hedge_ratio'].apply(lambda x: np.sum(x))
# filtered_df.query('abs(hdiff) <= 10')

# add 1 column tradingview_symbol that combine: hedge_ratio[0] * str(sid_1 + 'USDT.P') - str(sid_2 + 'USDT.P') * hedge_ratio[1]
filtered_df['tradingview_symbol'] = filtered_df.apply(lambda x: f'BINANCE:{x.sid_1}USDT.P * {x.hedge_ratio[0]} + BINANCE:{x.sid_2}USDT.P * {x.hedge_ratio[1]}', axis=1)

filtered_df.to_csv('pairs.csv', index=False)
filtered_df

,i,pairs,half_life,hedge_ratio,sid_1,sid_2,tradingview_symbol
1,1,"[LDO, DYDX]",16,"[4.491797448631916, -5.4009127311854055]",LDO,DYDX,BINANCE:LDOUSDT.P * 4.491797448631916 + BINANC...
2,2,"[AAVE, DYDX]",24,"[5.3840570405874075, -3.443989781899418]",AAVE,DYDX,BINANCE:AAVEUSDT.P * 5.3840570405874075 + BINA...
0,0,"[UNI, AAVE]",34,"[4.789300690919915, -4.882485457877565]",UNI,AAVE,BINANCE:UNIUSDT.P * 4.789300690919915 + BINANC...
